In [1]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# download yolo
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt # install dependencies (ignore errors)
# download my moedels
%cd yolov5/
!bash /content/yolov5/weights/download_weights.sh

Cloning into 'yolov5'...
remote: Enumerating objects: 3090, done.
remote: Total 3090 (delta 0), reused 0 (delta 0), pack-reused 3090
Receiving objects: 100% (3090/3090), 6.70 MiB | 32.50 MiB/s, done.
Resolving deltas: 100% (2050/2050), done.
     |████████████████████████████████| 276kB 7.2MB/s 
/content/yolov5
100% 14.5M/14.5M [00:00<00:00, 19.1MB/s]

100% 41.9M/41.9M [00:01<00:00, 32.5MB/s]

100% 91.6M/91.6M [00:02<00:00, 38.6MB/s]

100% 170M/170M [00:04<00:00, 37.4MB/s]



In [3]:
%cd ..
!mkdir forVideo
!mkdir makeVideo

/content


In [4]:
# get every frame of my video

import cv2
vidcap = cv2.VideoCapture('/content/drive/MyDrive/tests/My images/VID_20201121_194002.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/forVideo/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [5]:
import os
from tqdm import tqdm

weightPath_traffic_signs = "/content/drive/My Drive/YOLO/costum Yolo/signs detection/yolo to traffiSignRec/best.pt"
weightPath_coco = "/content/yolov5/yolov5l.pt"
myDir = "/content/forVideo"

for img in tqdm(os.listdir(myDir), total=len(os.listdir(myDir))):
    myPath = os.path.join(myDir,img)
    output = !python /content/yolov5/detect.py --source "$myPath" --weights $weightPath_coco --save-txt


100%|██████████| 236/236 [26:35<00:00,  6.76s/it]


In [6]:
stop # na allaxw sigoura to amaxi average i to traffic light

NameError: ignored

In [19]:
items_width = {
    "0":  0.4,   # person
    "2":  1.3,   # car
    "4":  0.5,   # motorbike
    "6":  2.1,   # bus
    "8":  2,     # truck
    "9":  0.35,   # traffic light
    "12": 0.6,   # stop sign
    "16": 0.3,   # cat
    "17": 0.4,   # dog
    "18": 1,     # horse
    "19": 0.9,   # sheep
    "20": 0.63   # cow 
}
print(items_width["0"])

0.4


In [8]:
# Initialize focal lenght
KNOWN_DISTANCE = 1
# initialize the known object width, which in this case, the piece of
# paper is 12 inches wide
KNOWN_WIDTH = 0.55
# load the furst image that contains an object that is KNOWN TO BE 2 feet
# from our camera, then find the paper marker in the image, and initialize
# the focal length

img = cv2.imread("/content/forVideo/frame0.jpg")
imgHeight, imgWidth, channels = img.shape

monitorWidth = 0.4 *imgWidth

# F = (P x  D) / W  # P: width in image
focalLength = (monitorWidth * KNOWN_DISTANCE) / KNOWN_WIDTH

In [9]:
def distance_to_camera(knownWidth, focalLength, perWidth):
	# compute and return the distance from the maker to the camera
	return (knownWidth * focalLength) / perWidth

In [20]:
from pathlib import Path

detected_path = "/content/runs/detect"
for exps in tqdm(os.listdir(detected_path), total=len(os.listdir(detected_path))):
    expsPath = os.path.join(detected_path, exps) # /content/runs/detect/exp
    for expsInfo in os.listdir(expsPath):
        
        if expsInfo.endswith(".jpg"):
            test_image_path = os.path.join(expsPath, expsInfo)
            test_image = cv2.imread(test_image_path)
            cutLast4 = expsInfo.rsplit(".")
            # Check if labels exists
            labelsPath = f"{expsPath}/labels/{cutLast4[0]}.txt"
            my_file = Path(labelsPath)
            if my_file.is_file():
                f = open(labelsPath, "r")
                fileslines = f.read().splitlines()
                
                for line in fileslines:
                    info = line.rsplit(" ")
                    clas = info[0]
                    x_center = int(float(info[1]) * imgWidth)
                    y_center = int(float(info[2]) * imgHeight)

                    mywidth = float(info[3]) * imgWidth

                    
                    # Check if Key Exists
                    if "{}".format(clas) in items_width:
                        known_width = items_width["{}".format(clas)]        
                        p = float(mywidth)
                        distance = distance_to_camera(known_width, focalLength, p)
                        
                        cv2.putText(test_image, "%.2fm" %distance,
	                        (x_center, y_center), cv2.FONT_HERSHEY_SIMPLEX,
	                        1.0, (0, 255, 0), 3)
                        
                        # Move every image with all predictions in another folder
            
            #Save image
            cv2.imwrite(f"/content/makeVideo/{expsInfo}", test_image)

100%|██████████| 236/236 [00:06<00:00, 36.90it/s]


In [ ]:
stop

In [21]:
# make my video
fps = 10.0
image_folder = '/content/makeVideo'
video_name = 'distanceVideo.mp4'
num_of_images = os.listdir(image_folder)
num_of_images = len(num_of_images)
images = []
for _ in range(0, num_of_images, 1):
    images.append(f"{image_folder}/frame{_}.jpg")


frame = cv2.imread(images[0])
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter(video_name, fourcc, fps, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [ ]:
stop

In [ ]:
# move the video to any folder
!mv /content/video.mp4 "/content/drive/MyDrive/tests"


In [18]:
# clear my dir
directory = "/content/makeVideo"
for img in os.listdir(directory):
    curPath = f"{directory}/{img}"
    !rm $curPath